In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pygame, random

pygame.init()

# WINDOW
WINDOW_SIZE = 600
WINDOW = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
pygame.display.set_caption('Sorting Algorithm Visualization')

# VARIABLES
RECT_WIDTH = 20
clock = pygame.time.Clock()
FPS = 10

# COLORS
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
GREY = (170, 170, 170)
LIGHT_BLUE = (64, 224, 208)

# CLASSES
class Rectangle:
    def __init__(self, color, x, height):
        self.color = color
        self.x = x
        self.width = RECT_WIDTH
        self.height = height

    def select(self):
        self.color = BLUE

    def unselect(self):
        self.color = PURPLE

    def set_smallest(self):
        self.color = LIGHT_BLUE

    def set_sorted(self):
        self.color = GREEN

# FUNCTIONS
def create_rectangles():
    num_rectangles = WINDOW_SIZE // RECT_WIDTH - 5
    rectangles = []
    heights = []

    for i in range(5, num_rectangles):
        height = random.randint(20, 500)
        while height in heights:
            height = random.randint(20, 500)

        heights.append(height)
        rect = Rectangle(PURPLE, i * RECT_WIDTH, height)
        rectangles.append(rect)

    return rectangles

def draw_rects(rectangles):
    WINDOW.fill(GREY)

    for rect in rectangles:
        pygame.draw.rect(WINDOW, rect.color, (rect.x, WINDOW_SIZE - rect.height, rect.width, rect.height))
        pygame.draw.line(WINDOW, BLACK, (rect.x, WINDOW_SIZE), (rect.x, WINDOW_SIZE - rect.height))
        pygame.draw.line(WINDOW, BLACK, (rect.x + rect.width, WINDOW_SIZE), (rect.x + rect.width, WINDOW_SIZE - rect.height))
        pygame.draw.line(WINDOW, BLACK, (rect.x, WINDOW_SIZE - rect.height), (rect.x + rect.width, WINDOW_SIZE - rect.height))

# Sorting Algorithms
def selection_sort(rectangles):
    num_rectangles = len(rectangles)

    for i in range(num_rectangles):
        min_index = i
        rectangles[i].set_smallest()

        for j in range(i + 1, num_rectangles):
            rectangles[j].select()
            draw_rects(rectangles)

            if rectangles[j].height < rectangles[min_index].height:
                rectangles[min_index].unselect()
                min_index = j
                
            rectangles[min_index].set_smallest()
            draw_rects(rectangles)
            rectangles[j].unselect()

            yield

        rectangles[i].x, rectangles[min_index].x = rectangles[min_index].x, rectangles[i].x
        rectangles[i], rectangles[min_index] = rectangles[min_index], rectangles[i]

        rectangles[min_index].unselect()
        rectangles[i].set_sorted()

        draw_rects(rectangles)

def bubble_sort(rectangles):
    num_rectangles = len(rectangles)

    for i in range(num_rectangles):
        for j in range(0, num_rectangles - i - 1):
            rectangles[j].select()
            rectangles[j + 1].select()
            draw_rects(rectangles)

            if rectangles[j].height > rectangles[j + 1].height:
                rectangles[j].x, rectangles[j + 1].x = rectangles[j + 1].x, rectangles[j].x
                rectangles[j], rectangles[j + 1] = rectangles[j + 1], rectangles[j]

            yield

            rectangles[j].unselect()
            rectangles[j + 1].unselect()

        rectangles[num_rectangles - i - 1].set_sorted()

def insertion_sort(rectangles):
    num_rectangles = len(rectangles)

    for i in range(1, num_rectangles):
        key_rect = rectangles[i]
        j = i - 1

        while j >= 0 and key_rect.height < rectangles[j].height:
            rectangles[j + 1] = rectangles[j]
            rectangles[j + 1].x = rectangles[j].x
            j -= 1
            draw_rects(rectangles)
            yield

        rectangles[j + 1] = key_rect
        rectangles[j + 1].x = (j + 1) * RECT_WIDTH
        draw_rects(rectangles)

        key_rect.set_sorted()

# Merge Sort
def merge_sort(rectangles, l=0, r=None):
    if r is None:
        r = len(rectangles) - 1

    if l < r:
        m = (l + r) // 2
        yield from merge_sort(rectangles, l, m)
        yield from merge_sort(rectangles, m + 1, r)
        yield from merge(rectangles, l, m, r)

def merge(rectangles, l, m, r):
    left = rectangles[l:m + 1]
    right = rectangles[m + 1:r + 1]

    i = j = 0
    k = l

    while i < len(left) and j < len(right):
        if left[i].height <= right[j].height:
            rectangles[k] = left[i]
            left[i].x = k * RECT_WIDTH
            i += 1
        else:
            rectangles[k] = right[j]
            right[j].x = k * RECT_WIDTH
            j += 1
        draw_rects(rectangles)
        yield
        k += 1

    while i < len(left):
        rectangles[k] = left[i]
        left[i].x = k * RECT_WIDTH
        i += 1
        draw_rects(rectangles)
        yield
        k += 1

    while j < len(right):
        rectangles[k] = right[j]
        right[j].x = k * RECT_WIDTH
        j += 1
        draw_rects(rectangles)
        yield
        k += 1

# Quick Sort
def quick_sort(rectangles, low=0, high=None):
    if high is None:
        high = len(rectangles) - 1

    if low < high:
        pi = yield from partition(rectangles, low, high)
        yield from quick_sort(rectangles, low, pi - 1)
        yield from quick_sort(rectangles, pi + 1, high)

def partition(rectangles, low, high):
    pivot = rectangles[high]
    i = low - 1

    for j in range(low, high):
        rectangles[j].select()
        pivot.select()
        draw_rects(rectangles)
        yield

        if rectangles[j].height < pivot.height:
            i += 1
            rectangles[i], rectangles[j] = rectangles[j], rectangles[i]
            rectangles[i].x, rectangles[j].x = rectangles[j].x, rectangles[i].x
            draw_rects(rectangles)
            yield

        rectangles[j].unselect()

    rectangles[i + 1], rectangles[high] = rectangles[high], rectangles[i + 1]
    rectangles[i + 1].x, rectangles[high].x = rectangles[high].x, rectangles[i + 1].x

    draw_rects(rectangles)
    yield

    return i + 1

def display_text(txt, y, size):
    FONT = pygame.font.SysFont('Futura', size)

    text = FONT.render(txt, True, BLACK)
    text_rect = text.get_rect(center=(WINDOW_SIZE / 2, y))
    WINDOW.blit(text, text_rect)

# MAIN FUNCTION
def main():
    rectangles = create_rectangles()
    draw_rects(rectangles)
    sorting_generator = None
    selected_algorithm = 'Selection Sort'

    # MAIN LOOP
    run = True
    sorting = False
    while run:
        clock.tick(FPS)

        if sorting and sorting_generator is not None:
            try:
                next(sorting_generator)
            except StopIteration:
                sorting = False
        else:
            draw_rects(rectangles)

        display_text(f'Sorting Algorithm: {selected_algorithm}', 30, 40)
        display_text('Press SPACE to start sorting or pause, Q to quit.', 70, 30)
        display_text('1: Selection Sort | 2: Bubble Sort | 3: Insertion Sort', 110, 30)
        display_text('4: Merge Sort | 5: Quick Sort', 150, 30)

        # EVENT HANDLER
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    sorting = not sorting
                    if sorting:
                        rectangles = create_rectangles()  # Reset rectangles when sorting starts
                        if selected_algorithm == 'Selection Sort':
                            sorting_generator = selection_sort(rectangles)
                        elif selected_algorithm == 'Bubble Sort':
                            sorting_generator = bubble_sort(rectangles)
                        elif selected_algorithm == 'Insertion Sort':
                            sorting_generator = insertion_sort(rectangles)
                        elif selected_algorithm == 'Merge Sort':
                            sorting_generator = merge_sort(rectangles)
                        elif selected_algorithm == 'Quick Sort':
                            sorting_generator = quick_sort(rectangles)
                if event.key == pygame.K_q:
                    run = False
                if event.key == pygame.K_1:
                    selected_algorithm = 'Selection Sort'
                if event.key == pygame.K_2:
                    selected_algorithm = 'Bubble Sort'
                if event.key == pygame.K_3:
                    selected_algorithm = 'Insertion Sort'
                if event.key == pygame.K_4:
                    selected_algorithm = 'Merge Sort'
                if event.key == pygame.K_5:
                    selected_algorithm = 'Quick Sort'

        pygame.display.update()

    pygame.quit()

main()
